In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

In [2]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

In [3]:
df_elec1 = df_elec.pivot(index='DateHour', columns='Minute', values='Value')
df_elec1 = df_elec1.reset_index() # DateHour를 index에 두지 않고, Column에 두어서 처리해봅니다.
df_elec1

Minute,DateHour,15분,30분,45분,60분
0,2021-01-01 00:00:00,62,61,61,61
1,2021-01-01 01:00:00,96,93,116,113
2,2021-01-01 02:00:00,106,96,106,107
3,2021-01-01 03:00:00,92,110,110,109
4,2021-01-01 04:00:00,108,105,106,108
...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139
6164,2021-09-14 20:00:00,124,130,128,130
6165,2021-09-14 21:00:00,134,130,125,124
6166,2021-09-14 22:00:00,100,109,120,114


In [4]:
# 휴일을 저장합니다. (date 형식)
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, # pd.Series.dt accesor의 weekday를 이용합니다.
    Hour = df_elec1['DateHour'].dt.hour, 
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'), # 실제 시험에도 변수명과 내용이 일치하지 않을 수 있습니다. 유의 하세요
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn= df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # pd.Series.dt.date는 pd.Series를 반환합니다. pd.Series.isin을 사용합니다.
    Avg = df_elec1.mean(axis=1), # 각 행별 15분, 30분, 45분, 60분의 대한 평균입니다.
    TotalHour = df_elec1.sum(axis=1) # 각 행별 15분, 30분, 45분, 60분의 대한 합계입니다.
)
df_elec1

Minute,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427
...,...,...,...,...,...,...,...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139,1,19,1,0,0,153.25,613
6164,2021-09-14 20:00:00,124,130,128,130,1,20,1,0,0,128.00,512
6165,2021-09-14 21:00:00,134,130,125,124,1,21,1,0,0,128.25,513
6166,2021-09-14 22:00:00,100,109,120,114,1,22,1,0,0,110.75,443


In [5]:
df_info1 = df_info.fillna(0)

In [6]:
# merge를 하면 기준이 인덱스에 있을 경우에 인덱스에 있었던, DateHour는 사라지지만
# 컬럼에 있는 DateHour는 남아 있습니다.
df_basetable1 = df_elec1.merge(df_info1, left_on='DateHour', right_on='Date', how='inner')
df_basetable1

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,Date,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,2021-01-01 00:00:00,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,2021-01-01 01:00:00,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,2021-01-01 02:00:00,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,2021-01-01 03:00:00,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,2021-01-01 04:00:00,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139,1,19,1,0,0,...,613,2021-09-14 19:00:00,1497,21.7,3.6,85,9.4,167.2,2.442088,1.5
6164,2021-09-14 20:00:00,124,130,128,130,1,20,1,0,0,...,512,2021-09-14 20:00:00,45,22.2,4.2,78,9.4,167.2,0.087891,1.5
6165,2021-09-14 21:00:00,134,130,125,124,1,21,1,0,0,...,513,2021-09-14 21:00:00,149,22.2,4.3,76,9.4,167.2,0.290448,1.5
6166,2021-09-14 22:00:00,100,109,120,114,1,22,1,0,0,...,443,2021-09-14 22:00:00,66,22.0,2.5,79,9.4,167.2,0.148984,1.5


In [7]:
df_prob3 = df_basetable1.copy()
# DateHour가 인덱스에 있을 경우에는 target을 대입할 때, index를 기준으로 대입이 되어 인덱스를 변경하여 target을 생성할 수 있었지만,
# DateHour가 컬럼에 있을 경우에는 join을 이용해 매칭을 하는 작업을 합니다.
df_prob3['target'] = df_prob3[['DateHour']].join(
    pd.Series(
        df_prob3['TotalHour'].values,
        index=df_prob3['DateHour'] - pd.Timedelta(days=1),
        name='target'
    ), on='DateHour'
)['target']
df_prob3 = df_prob3.dropna().copy()

In [8]:
df_prob4 = pd.concat(
    [df_prob3] +
    [df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)], 
    axis=1
).dropna()
df_prob4

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,전기요금(계절),공장인원,인건비,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6
144,2021-01-07 00:00:00,22,22,22,25,3,0,0,0,0,...,109.8,0.000000,1.5,252.0,96.0,96.0,271.0,88.0,253.0,245.0
145,2021-01-07 01:00:00,26,22,22,22,3,1,0,0,0,...,109.8,0.369565,1.5,396.0,85.0,85.0,432.0,99.0,418.0,418.0
146,2021-01-07 02:00:00,22,21,21,22,3,2,0,0,0,...,109.8,0.000000,1.5,411.0,85.0,85.0,439.0,88.0,415.0,415.0
147,2021-01-07 03:00:00,22,22,22,23,3,3,0,0,0,...,109.8,0.629213,1.5,398.0,90.0,90.0,426.0,88.0,421.0,421.0
148,2021-01-07 04:00:00,22,26,26,23,3,4,0,0,0,...,109.8,2.865979,1.5,420.0,93.0,93.0,435.0,95.0,427.0,427.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,167.2,4.763514,1.5,613.0,92.0,101.0,567.0,569.0,601.0,609.0
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,167.2,0.028986,1.5,512.0,93.0,93.0,464.0,513.0,413.0,535.0
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,167.2,0.146939,1.5,513.0,100.0,93.0,466.0,510.0,487.0,524.0
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,167.2,0.025000,1.5,443.0,92.0,93.0,409.0,437.0,422.0,461.0


In [9]:
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < '2021-08-14'].copy()
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= '2021-08-14'].copy()

In [10]:
# 실제 상황임을 가정하여 df_prob6_train만 정답을 알고 있다고 생각하여 진행합니다.
# 실제 학습셋과 검증셋을 만듭니다. 2021-07-14 기준
df_train = df_prob6_train[df_prob6_train['DateHour'] < '2021-07-14'].copy()
df_test = df_prob6_train[df_prob6_train['DateHour'] >= '2021-07-14'].copy()

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

In [12]:
# 먼저 baseline 모델을 만듭니다.
# 문제에서 가장 좋았던 모델을 사용해봅니다.
from sklearn.svm import SVR

reg_svr = make_pipeline(
    StandardScaler(), SVR(C=10, kernel='rbf', gamma=0.1)
)
X_svr = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
reg_svr.fit(df_train[X_svr], df_train['target'])

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [13]:
(
    mean_absolute_error(df_train['target'], reg_svr.predict(df_train[X_svr])),
    mean_absolute_error(df_test['target'], reg_svr.predict(df_test[X_svr]))
)

(61.50534596142382, 148.65000821712516)

In [14]:
# 제출결과는 전체 학습셋으로 모델을 학습합니다.
reg_svr.fit(df_prob6_train[X_svr], df_prob6_train['target'])

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [15]:
# Baseline의 결과 파일을 만듭니다.
# 방법 1. 인덱스 활용
pd.DataFrame(
    reg_svr.predict(df_prob6_test[X_svr]),
    index = df_prob6_test['DateHour'],
    columns=['TotalHour']
).to_csv('answer6.csv')

In [16]:
# Baseline의 결과 파일을 만듭니다.
# 방법 2. 컬럼만을 활용
pd.DataFrame(
    {
        'DateHour': df_prob6_test['DateHour'],
        'TotalHour': reg_svr.predict(df_prob6_test[X_svr]),
    }
).to_csv('answer6.csv', index=None)

In [17]:
mean_absolute_error(
    df_prob6_test['target'],
    reg_svr.predict(df_prob6_test[X_svr])
)

30.017658858928023

In [18]:
# 다른 모델을 만들어 봅니다.

from sklearn.ensemble import GradientBoostingRegressor
X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]

reg_gb = GradientBoostingRegressor(
    loss='huber', # 손실함수 huber
    n_estimators=75, # 트리의 수 75
    max_depth=7, # 트리의 최대 깊이 7
    min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수 4
    random_state=123 # 랜덤 시드 123
)
reg_gb.fit(df_train[X_gb], df_train['target'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='huber', max_depth=7,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=4, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=75,
                          n_iter_no_change=None, presort='auto',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
(
    mean_absolute_error(df_train['target'], reg_gb.predict(df_train[X_gb])),
    mean_absolute_error(df_test['target'], reg_gb.predict(df_test[X_gb]))
)

(36.73714649971901, 117.32391481946111)

In [20]:
# Baseline 보다는 좋은 성능을 보입니다.
reg_gb.fit(df_prob6_train[X_gb], df_prob6_train['target'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='huber', max_depth=7,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=4, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=75,
                          n_iter_no_change=None, presort='auto',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
mean_absolute_error(
    df_prob6_test['target'],
    reg_gb.predict(df_prob6_test[X_gb])
)

38.86815607791259

In [22]:
# 결과를 GBDT로 바꿉니다.
reg_gb.fit(df_prob6_train[X_gb], df_prob6_train['target'])
pd.DataFrame(
    reg_gb.predict(df_prob6_test[X_gb]),
    index = df_prob6_test['DateHour'],
    columns=['TotalHour']
).to_csv('answer6.csv')

In [23]:
# 두 모델을 Voting 모델을 사용하여, ensemble 모델을 구성합니다.
from sklearn.ensemble import VotingRegressor
reg_vt = VotingRegressor([
    ('svr', reg_svr),
    ('gb', reg_gb)
])
X_vt = np.unique(X_gb + X_svr).tolist()
reg_vt.fit(df_train[X_vt], df_train['target'])

VotingRegressor(estimators=[('svr',
                             Pipeline(memory=None,
                                      steps=[('standardscaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('svr',
                                              SVR(C=10, cache_size=200,
                                                  coef0=0.0, degree=3,
                                                  epsilon=0.1, gamma=0.1,
                                                  kernel='rbf', max_iter=-1,
                                                  shrinking=True, tol=0.001,
                                                  verbose=False))],
                                      verbose=False)),
                            ('gb',
                             GradientBoo

In [24]:
# 성능의 향상은 보이지 않습니다 - 버립니다.
(
    mean_absolute_error(df_train['target'], reg_vt.predict(df_train[X_vt])),
    mean_absolute_error(df_test['target'], reg_vt.predict(df_test[X_vt]))
)

(44.51545975069041, 146.8097471782379)

In [25]:
# 그래도 결과는 확인해봅니다.
reg_vt.fit(df_prob6_train[X_vt], df_prob6_train['target'])
mean_absolute_error(
    df_prob6_test['target'],
    reg_vt.predict(df_prob6_test[X_vt])
)

46.90391226382624